# Outlier Detection

This notebook will walk you through a workflow for outlier detection.

## TOC

 * [Detecting outlier jobs](#detect-outlier-jobs-study-1)
  * [partition jobs](#partition-jobs)
  * [create a trained model](#create-ref-model)
 * [Root Cause Analysis (RCA)](#rca-job)
 * [Detecting outlier operations](#detect-outlier-ops)
  * [Root Cause Analysis for Ops](#rca-ops)
 * [Notes and areas for investigation](#notes)

## <a name="detect-outlier-jobs-study-1">Case Study 1 - Detecting outlier jobs</a>

In this example we will use synthetic data. The data was generated using:
```
sample/outlier/workload.sh
```

The script basically compiles the linux kernel a few times. For certain compiles
it adds a background workload, so the compile takes longer. The slower job is marked
with the an 'outlier' suffix. However, we will pretend we don't know the outlier
and figure it out.

Along the way, we will also learn how to create a trained model, and use it 
for future outlier detection.


### Requirements

You will need to import data lying in `sample/outlier/*.tgz`:
```
$ ./epmt -v submit sample/outlier/*.tgz
```

In [1]:
import epmt_query as eq
import epmt_outliers as eod

{'provider': 'postgres', 'host': 'localhost', 'dbname': 'EPMT', 'user': 'postgres', 'password': 'example'}


In [2]:
jobs = eq.get_jobs(tag='exp_name:linux_kernel', fmt='terse')
jobs

['kern-6656-20190614-185359',
 'kern-6656-20190614-190245',
 'kern-6656-20190614-191138',
 'kern-6656-20190614-192044-outlier',
 'kern-6656-20190614-194024',
 'kern-6656-20190614-194953',
 'kern-6656-20190614-195909',
 'kern-6656-20190614-200819',
 'kern-6656-20190614-201744-outlier']

In [3]:
# As a first pass let's see whether the outliers can be auto-detected
(df, fdict) = eod.detect_outlier_jobs(jobs)
df

,jobid,duration,cpu_time,num_procs
0,kern-6656-20190614-185359,0,1,0
1,kern-6656-20190614-190245,0,0,0
2,kern-6656-20190614-191138,0,0,0
3,kern-6656-20190614-192044-outlier,1,1,0
4,kern-6656-20190614-194024,0,0,0
5,kern-6656-20190614-194953,0,0,0
6,kern-6656-20190614-195909,0,0,0
7,kern-6656-20190614-200819,0,0,0
8,kern-6656-20190614-201744-outlier,1,1,0


As you can see, while we did catch both the outliers, there
is also the "false positive" on one "non-outlier" process
The reason the 1 is marked for `duration` and `cpu_time` but `not num_procs`
is because the background compute process increased the job duration
but not the number of sub-processes of our workload.

In [4]:
fdict

{'cpu_time': ({'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-185359',
   'kern-6656-20190614-192044-outlier',
   'kern-6656-20190614-201744-outlier'}),
 'duration': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-192044-outlier', 'kern-6656-20190614-201744-outlier'}),
 'num_procs': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-192044-outlier',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819',
   'kern-6656-20190614-201744-outlier'},
  set())}

`fdict` the other return value is a dictionary keyed by `feature`. The value is a tuple of two partitions based on the `feature`. The first partition being the reference set, and the second partition is the outlier set.
This partitioning <a name="partition-jobs"></a> can be more simply obtained as follows:

In [5]:
parts = eod.partition_jobs(jobs, features=['duration'])
parts

{'duration': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-192044-outlier', 'kern-6656-20190614-201744-outlier'})}

Above, we just got the partitioning of the jobs on a single `feature` -- `duration`.

Now would be a good time to <a name="create-ref-model"></a>create a trained model based on the
set of jobs in the reference partition:

In [6]:
ref_jobs = parts['duration'][0]
ref_jobs

{'kern-6656-20190614-185359',
 'kern-6656-20190614-190245',
 'kern-6656-20190614-191138',
 'kern-6656-20190614-194024',
 'kern-6656-20190614-194953',
 'kern-6656-20190614-195909',
 'kern-6656-20190614-200819'}

In [7]:
r = eq.create_refmodel(ref_jobs, tag='exp_name:linux_kernel;type:ref')

In [8]:
r['id'], r['tags']

(9, {'exp_name': 'linux_kernel', 'type': 'ref'})

We added a tag to help search for this trained/ref model later.

In [9]:
# using the trained model is as simple as:
(df, _) = eod.detect_outlier_jobs(jobs, trained_model = r['id'])
df

,jobid,duration,cpu_time,num_procs
0,kern-6656-20190614-185359,0,0,0
1,kern-6656-20190614-190245,0,0,0
2,kern-6656-20190614-191138,0,0,0
3,kern-6656-20190614-192044-outlier,1,1,0
4,kern-6656-20190614-194024,0,0,0
5,kern-6656-20190614-194953,0,0,0
6,kern-6656-20190614-195909,0,0,0
7,kern-6656-20190614-200819,0,0,0
8,kern-6656-20190614-201744-outlier,1,1,0


Obviously the jobs that were used to create the reference model will not be 
classifed as outliers for any feature.

This marks the end of this case study. In a following study we will explore how
to detect outliers in individual operations and create a trained model for ops.

## <a name="rca-job">Case Study II - Root Cause Analysis</a>

In this study we will do an RCA with real data generated from GFDL PP runs.


### Requirements
Please import the following data:

```
$ ./epmt -v submit $(cat <<EOT
> sample/ppr-batch/1854/625151.tgz
> sample/ppr-batch/1859/627907.tgz
> sample/ppr-batch/1869/633114.tgz
> sample/ppr-batch/1864/629322.tgz
> sample/ppr-batch/1884/685001.tgz
> sample/ppr-batch/1874/675992.tgz
> sample/ppr-batch/1879/680163.tgz
> sample/ppr-batch/1889/691209.tgz
> sample/ppr-batch/1894/693129.tgz
> EOT
> )
```

All these jobs share the following tags: `{u'ocn_res': u'0.5l75', u'atm_res': u'c96l49', u'exp_component': u'ocean_annual_z_1x1deg', u'exp_name': u'ESM4_historical_D151'}`. The difference is only that they have different values for `('exp_time', 'script_name')`.

If you are curious how we found these comparable jobs, here is the query:
```
>>> x = Job.select(lambda j: j.tags['exp_component'] == 'ocean_annual_z_1x1deg').filter(lambda j: j.tags['exp_name'] == 'ESM4_historical_D151')
>>> eq.get_jobs(x, fmt="terse")
```

In [10]:
jobs = eq.get_jobs(tag="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fmt='terse')
jobs

['625151',
 '627907',
 '633114',
 '629322',
 '685001',
 '675992',
 '680163',
 '802938',
 '691209',
 '693129',
 '696110',
 '804266']

Now, let's partition the jobs by `cpu_time`: 

In [11]:
parts = eod.partition_jobs(jobs, features=['cpu_time'])
parts

{'cpu_time': ({'627907',
   '629322',
   '633114',
   '675992',
   '680163',
   '685001',
   '691209',
   '693129',
   '696110',
   '802938',
   '804266'},
  {'625151'})}

As you can see, the first partition contains 9 conformant jobs, and the outlier
partition contains a single job `625151`. Let's see how different is the `cpu_time`
of this job v. the rest:

In [12]:
jobs_df = eq.get_jobs(jobs, fmt='pandas', order='desc(j.cpu_time)')
display(jobs_df.columns.values)
jobs_df[['jobid', 'cpu_time', 'duration', 'num_procs']]

array(['PERF_COUNT_SW_CPU_CLOCK', 'account', 'all_proc_tags',
       'cancelled_write_bytes', 'cpu_time', 'created_at',
       'delayacct_blkio_time', 'duration', 'end', 'env_changes_dict',
       'env_dict', 'exitcode', 'guest_time', 'inblock', 'info_dict',
       'invol_ctxsw', 'jobid', 'jobname', 'jobscriptname', 'majflt',
       'minflt', 'num_procs', 'num_threads', 'outblock', 'processor',
       'queue', 'rchar', 'rdtsc_duration', 'read_bytes', 'rssmax',
       'sessionid', 'start', 'submit', 'syscr', 'syscw', 'systemtime',
       'tags', 'time_oncpu', 'time_waiting', 'timeslices', 'updated_at',
       'user', 'user+system', 'usertime', 'vol_ctxsw', 'wchar',
       'write_bytes'], dtype=object)

,jobid,cpu_time,duration,num_procs
0,625151,1224444629,10425623185,13530
1,627907,694594906,6589174875,4411
2,629322,622137956,7286331754,4411
3,802938,558475582,3986871458,4411
4,691209,536679993,860163243,4411
5,804266,535237219,3285839213,4411
6,693129,533686454,3619324767,4411
7,675992,520837207,9114150525,4411
8,685001,480310481,6815710476,4428
9,680163,462936089,6156192011,4411


It's really interesting that the outlier has thrice the number of processes
as the others. Let's see if we can uncover more..

In [13]:
(refs, outl) = parts['cpu_time']
(refs, outl)

({'627907',
  '629322',
  '633114',
  '675992',
  '680163',
  '685001',
  '691209',
  '693129',
  '696110',
  '802938',
  '804266'},
 {'625151'})

In [14]:
(_, df, flist) = eod.detect_rootcause(refs, '625151')
df

,num_procs,cpu_time,duration
count,11.000000,1.100000e+01,1.100000e+01
mean,4415.636364,5.252580e+08,5.205965e+09
std,10.993386,8.507230e+07,2.348081e+09
min,4411.000000,3.757608e+08,8.601632e+08
25%,4411.000000,4.716233e+08,3.567232e+09
50%,4411.000000,5.336865e+08,6.036720e+09
75%,4411.000000,5.475778e+08,6.702443e+09
max,4445.000000,6.945949e+08,9.114151e+09
input,13530.000000,1.224445e+09,1.042562e+10
ref_max_modified_z_score,4.946300,2.033400e+00,1.703300e+00


The features are ranked from the one with the highest score (w.r.to ref) from left to right, 
in decreasing importance.

In [15]:
flist

[('num_procs', 268.20769059701195),
 ('cpu_time', 4.2928100718009246),
 ('duration', 0.8478835202254447)]

If you would like to expand the RCA to other `features`, do as below:

In [16]:
(_, df, flist) = eod.detect_rootcause(refs, '625151', features=[])
df

,wchar,syscw,rchar,num_threads,num_procs,rssmax,syscr,vol_ctxsw,timeslices,minflt,...,invol_ctxsw,inblock,read_bytes,cancelled_write_bytes,majflt,rdtsc_duration,exitcode,processor,delayacct_blkio_time,guest_time
count,1.100000e+01,11.000000,1.100000e+01,11.000000,11.000000,1.100000e+01,11.000000,11.000000,11.000000,11.000000,...,11.000000,11.000000,1.100000e+01,1.100000e+01,11.000000,1.100000e+01,11,11,11,11
mean,7.272863e+10,4052543.727273,9.568864e+10,4680.909091,4415.636364,6.966260e+07,12201015.636364,821240.090909,846691.636364,7501751.636364,...,20757.454545,17971684.363636,9.201505e+09,3.074751e+09,458.909091,-7.355076e+15,0,0,0,0
std,8.439920e+04,274.828343,2.366415e+06,11.640056,10.993386,2.389784e+05,4829.967956,17789.607750,25269.441190,190515.545830,...,18702.991634,24344060.897759,1.246416e+10,3.940853e+09,1106.925332,1.739965e+16,0,0,0,0
min,7.272851e+10,4052375.000000,9.568720e+10,4676.000000,4411.000000,6.927494e+07,12198338.000000,802024.000000,820315.000000,7328295.000000,...,12130.000000,6736.000000,3.448832e+06,3.644375e+08,2.000000,-5.328561e+16,0,0,0,0
25%,7.272856e+10,4052427.000000,9.568736e+10,4676.000000,4411.000000,6.957008e+07,12198804.500000,812348.500000,829796.000000,7367003.500000,...,13272.500000,193036.000000,9.883443e+07,2.222774e+09,10.500000,1.964003e+13,0,0,0,0
50%,7.272863e+10,4052464.000000,9.568772e+10,4676.000000,4411.000000,6.963037e+07,12198986.000000,813963.000000,836282.000000,7514178.000000,...,14236.000000,7361584.000000,3.769131e+09,2.236223e+09,62.000000,3.109937e+13,0,0,0,0
75%,7.272871e+10,4052499.000000,9.568820e+10,4676.000000,4411.000000,6.967425e+07,12200396.500000,822491.000000,856153.000000,7545145.500000,...,18125.500000,24766436.000000,1.268042e+10,2.236244e+09,302.000000,5.086433e+13,0,0,0,0
max,7.272873e+10,4053353.000000,9.569491e+10,4712.000000,4445.000000,7.008892e+07,12214610.000000,862453.000000,895097.000000,8003186.000000,...,76421.000000,78131160.000000,4.000315e+10,1.478598e+10,3750.000000,7.181059e+13,0,0,0,0
input,1.388007e+11,4436801.000000,9.854065e+10,14551.000000,13530.000000,1.336608e+08,13084676.000000,3150991.000000,3198404.000000,20550722.000000,...,32844.000000,25021616.000000,1.281107e+10,2.331615e+09,80.000000,1.009561e+14,0,0,0,0
ref_max_modified_z_score,8.902000e-01,16.206200,1.295190e+01,4.946300,4.946300,3.739400e+00,28.027600,9.516000,5.303600,2.377300,...,32.314200,6.490200,6.490200e+00,3.444340e+05,46.065900,1.897052e+03,0,0,0,0


As you can see, the `modified_z_score_ratio` for `rssmax` is 139. This shows that even the memory
footprint of the outlier is much more than those of the reference jobs.

In our next study we will attempt to determine which `operations` from within the jobs were outliers.

## <a name="detect-outlier-ops">Case Study III - Detect Outlier Operations</a>

This study shows how we can find outlier operations across a set of jobs.

Please [review the previous study](#rca-job) as this has the same requirements.

In [17]:
jobs = eq.get_jobs(tag="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fmt='terse')

In [18]:
len(jobs)

12

In [36]:
(df, parts, scores_df, sorted_tags, sorted_features) = eod.detect_outlier_ops(jobs)
df.head(20)

,jobid,tags,duration,cpu_time,num_procs
0,627907,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",0,0,0
1,629322,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",0,1,0
2,633114,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",0,0,0
3,675992,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",0,1,0
4,680163,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",0,1,0
5,685001,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",1,1,0
6,691209,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",0,0,0
7,693129,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",0,0,0
8,696110,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",0,1,0
9,802938,"{'op': 'mv', 'op_instance': '13', 'op_sequence...",0,0,0


In [20]:
len(sorted_tags)

416

As you can see there are 416 unique operations. Since we want to focus our attention
on the operations, that have the highest deviation across jobs, `detect_outlier_ops`
helps us by ordering the dataframe, `df` in the order of descreasing operation (tag)
importance. To figure out the importance of a tag, it uses the maximum of the scores
across all jobs, across all features for that particular tag. You can see the scores
ordered by decreasing tag importance in `scores_df`:

In [21]:
scores_df.head()

,tags,duration,cpu_time,num_procs
0,"{""op"": ""mv"", ""op_instance"": ""13"", ""op_sequence...",11.530,4043.151,0
1,"{""op"": ""mv"", ""op_instance"": ""10"", ""op_sequence...",1621.426,1.547,0
2,"{""op"": ""hsmget"", ""op_instance"": ""6"", ""op_seque...",824.428,0.973,0
3,"{""op"": ""hsmget"", ""op_instance"": ""7"", ""op_seque...",393.766,1.160,0
4,"{""op"": ""hsmget"", ""op_instance"": ""6"", ""op_seque...",387.099,0.000,0


The ordered list of tags (in decreasing importance) is also available in `sorted_tags`.
Let's see the top-5 tags. These could give a hint on which operations to focus on:

In [22]:
sorted_tags[:5]

[{'op': 'mv', 'op_instance': '13', 'op_sequence': '69'},
 {'op': 'mv', 'op_instance': '10', 'op_sequence': '60'},
 {'op': 'hsmget', 'op_instance': '6', 'op_sequence': '21'},
 {'op': 'hsmget', 'op_instance': '7', 'op_sequence': '19'},
 {'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}]

`detect_outlier_ops` thus already helps with RCA by ordering the output
in decreasing tag importance. It goes even further, by presenting a ordered
`features` list, in descreasing order of feature importance. Here the
importance of a `feature` is determined by summing the scores of the
feature across all tags. 

In [23]:
sorted_features

['duration', 'cpu_time', 'num_procs']

The full dataframe of scores of tags by feature, and ordered in
decreasing tag importance is available in `scores_df`, as shown earlier.

<a name="rca-ops"></a>Once you have the ordered list of tags (operations), you may want to do
further RCA analysis for a specific operation. To do so, you now use a 
*different* partitioning of jobs, since you need to specify the outlier
job for the specific operation that you care about.

Let's suppose we care about the top operation: `{'op': 'mv', 'op_instance': '13', 'op_sequence': '69'}`.
When we see the jobid `675992` was one of the outliers for this operation in `df`.

In [50]:
# first derive the list of jobs other than the outlier
jobs = eq.get_jobs(tag="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fltr='j.jobid != "675992"')
len(jobs)

11

In [51]:
(ret, df_rca, feature_scores) = eod.detect_rootcause_op(jobs, '675992', {'op': 'mv', 'op_instance': '13', 'op_sequence': '69'})
ret

True

In [52]:
df_rca

,cpu_time,duration,num_procs
count,10.000000,10.000000,10
mean,5598.100000,126.600000,1
std,843.195041,33.833580,0
min,4998.000000,99.000000,1
25%,4998.000000,114.250000,1
50%,4998.500000,115.500000,1
75%,5998.000000,122.500000,1
max,6998.000000,220.000000,1
input,10997.000000,126.000000,1
ref_max_modified_z_score,2697.325500,15.663400,0


In [53]:
# to get the full feature list, just pass features=[] to the same function
(ret, df_rca, feature_scores) = eod.detect_rootcause_op(jobs, '675992', {'op': 'mv', 'op_instance': '13', 'op_sequence': '69'}, features=[])
ret

True

In [54]:
df_rca

,exclusive_cpu_time,cpu_time,user+system,usertime,time_oncpu,rssmax,rchar,systemtime,minflt,duration,...,wchar,syscw,delayacct_blkio_time,syscr,vol_ctxsw,write_bytes,outblock,num_tids,cancelled_write_bytes,guest_time
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10,10,10,10.000000,10,10,10,10,10,10
mean,5598.100000,5598.100000,5598.100000,3399.000000,6144508.100000,3857.200000,45489.900000,2199.100000,1480.600000,126.600000,...,151,1,0,78.400000,5,4096,8,1,0,0
std,843.195041,843.195041,843.195041,966.091783,818528.984239,29.153997,80.268231,1135.077134,58.164900,33.833580,...,0,0,0,1.264911,0,0,0,0,0,0
min,4998.000000,4998.000000,4998.000000,1999.000000,5302080.000000,3844.000000,45387.000000,0.000000,1432.000000,99.000000,...,151,1,0,78.000000,5,4096,8,1,0,0
25%,4998.000000,4998.000000,4998.000000,2999.000000,5585648.250000,3848.000000,45484.000000,1999.000000,1433.000000,114.250000,...,151,1,0,78.000000,5,4096,8,1,0,0
50%,4998.500000,4998.500000,4998.500000,3499.000000,5895626.000000,3848.000000,45486.000000,1999.000000,1454.500000,115.500000,...,151,1,0,78.000000,5,4096,8,1,0,0
75%,5998.000000,5998.000000,5998.000000,3999.000000,6332507.750000,3848.000000,45492.250000,2999.000000,1522.750000,122.500000,...,151,1,0,78.000000,5,4096,8,1,0,0
max,6998.000000,6998.000000,6998.000000,4999.000000,7676162.000000,3940.000000,45678.000000,3999.000000,1576.000000,220.000000,...,151,1,0,82.000000,5,4096,8,1,0,0
input,10997.000000,10997.000000,10997.000000,7998.000000,11229753.000000,3940.000000,45675.000000,2999.000000,1474.000000,126.000000,...,151,1,0,78.000000,5,4096,8,1,0,0
ref_max_modified_z_score,2697.325500,2697.325500,2697.325500,2.023500,3.003200,6.205400,25.900800,1.349000,3.642300,15.663400,...,0,0,0,6.745000,0,0,0,0,0,0


## <a name="notes">Notes and areas for further investigation</a>

 * When we do `detect_outlier_ops` we also want to know the top jobs (ranked) by tag.
   We need this for subsequent RCA partitioning.
 * The columns of `df` and `scores_df` should be ordered in the same order as `sorted_features`.
